#스트레스 지수 예측

나이, 직업, 수면 시간, 수면의 질 등 관련 데이터로부터 스트레스 지수를 예측하는, 다중 분류 문제를 해결해 봅시다.

본 과제는 2점 만점이며, 정확도(Accuracy)를 평가 지표로 사용합니다.

Baseline보다 높은 정확도 달성: 2.0점

Baseline의 정확도와 동일(소수 5 번째 자리까지)하거나, 낮은 정확도 달성: 1.0점

미제출: 0점

(주의) 허용할 수 없을 정도로 낮은 Accuracy로 기록된 것도 미제출로 간주될 수 있습니다.

현재 Baseline은 scikit-learn 라이브러리의 DecisionTreeClassifier 클래스를 활용해 도출됐습니다.

데이터 세트 분할 시, random_state 값은 42로 지정했습니다.

수업 시간에 다뤘던 Decision Tree 알고리즘 및 하이퍼파라미터와 관련된 내용을 복습하고, 과제 3을 해결해 봅시다.

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline

# 1) 데이터 불러오기
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission.csv")

# 타깃과 입력 분리
X = train.drop(columns=["Stress Level"])
y = train["Stress Level"]

# 2) 숫자/문자형 컬럼 자동 구분
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

print("문자형 컬럼:", list(cat_cols))
print("숫자형 컬럼:", list(num_cols))

# 3) 컬럼별 전처리 정의
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ("num", "passthrough", num_cols),
    ]
)

# 4) 전처리 + 의사결정나무 파이프라인
pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", DecisionTreeClassifier(random_state=42))
])

# 5) train/valid 분할 (random_state=42 고정)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 6) 하이퍼파라미터 그리드 (너무 많으면 느리니 적당히)
param_grid = {
    "clf__max_depth": [3, 5, 7, 9, None],
    "clf__min_samples_split": [2, 5, 10],
    "clf__min_samples_leaf": [1, 2, 4],
    "clf__criterion": ["gini", "entropy"],
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring="accuracy",
    cv=5,
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)

# 7) 검증 정확도 확인
val_pred = grid.predict(X_val)
val_acc = accuracy_score(y_val, val_pred)
print("Validation Accuracy:", val_acc)


문자형 컬럼: ['Gender', 'Occupation', 'BMI Category', 'Blood Pressure']
숫자형 컬럼: ['Age', 'Sleep Duration', 'Quality of Sleep', 'Physical Activity Level', 'Heart Rate', 'Daily Steps']
Best params: {'clf__criterion': 'entropy', 'clf__max_depth': 5, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2}
Validation Accuracy: 0.9666666666666667


In [7]:
# 8) 최적 모델로 전체 train 다시 학습
best_model = grid.best_estimator_
best_model.fit(X, y)

# 9) test.csv에 대한 예측
test_pred = best_model.predict(test)


In [8]:
df = pd.read_csv("submission.csv")

df.dropna(axis=1, inplace=True)

# 교수님이 말한 부분: np.arange(...) 대신 내가 만든 모델 예측값
y_pred = test_pred

df["Stress Level"] = y_pred
df.to_csv("new_submission.csv", index=False)
print("new_submission.csv 저장 완료")


new_submission.csv 저장 완료
